## MNIST Training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [2]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
role

'arn:aws:iam::475933981307:role/service-role/AmazonSageMaker-ExecutionRole-20180102T172706'

## Download training and test data

In [4]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [8]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist', bucket="sagemaker-aim418-cyrusvahid")

## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [6]:
!cat 'mnist.py'

from __future__ import print_function

import logging
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #


def train(current_host, channel_input_dirs, hyperparameters, hosts, num_gpus):
    # SageMaker passes num_cpus, num_gpus and other args we can use to tailor training to
    # the current container environment, but here we just use simple cpu context.
    ctx = mx.cpu()

    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = hyperparameters.get('batch_size', 100)
    epochs = hyperparameters.get('epochs', 10)
    learning_rate = hyperparameters.get('learning_rate', 0.1)
    momentum = hyperparameters.get('momentum', 0.9)
    log_interval = 

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [10]:
m = MXNet("mnist.py",
          role=role,
          train_instance_count=1,
          train_instance_type="ml.c4.xlarge",
          framework_version="1.2.1",
          output_path="s3://sagemaker-aim418-cyrusvahid/output/",
          hyperparameters={'batch_size': 100,
                         'epochs': 5,
                         'learning_rate': 0.1,
                         'momentum': 0.9, 
                         'log_interval': 100})

After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [11]:
m.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-11-15-14-37-03-872


2018-11-15 14:37:04 Starting - Starting the training job...
2018-11-15 14:37:06 Starting - Launching requested ML instances......
2018-11-15 14:38:32 Starting - Preparing the instances for training.........
2018-11-15 14:39:58 Downloading - Downloading input data
2018-11-15 14:39:58 Training - Training image download completed. Training in progress..
2018-11-15 14:39:58,384 INFO - root - running container entrypoint
2018-11-15 14:39:58,385 INFO - root - starting train task
2018-11-15 14:39:58,390 INFO - container_support.training - Training starting
2018-11-15 14:39:59,392 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/mxnet#updating-your-mxnet-training-script.#033
2018-1

[Epoch 11 Batch 300] Training: accuracy=0.993920, 4788.346234 samples/s
[Epoch 11 Batch 400] Training: accuracy=0.993691, 5640.158677 samples/s
[Epoch 11 Batch 500] Training: accuracy=0.992635, 5719.843445 samples/s
[Epoch 11] Training: accuracy=0.992000
[Epoch 11] Validation: accuracy=0.974100
[Epoch 12 Batch 100] Training: accuracy=0.994059, 6161.841661 samples/s
[Epoch 12 Batch 200] Training: accuracy=0.993483, 5508.673496 samples/s
[Epoch 12 Batch 300] Training: accuracy=0.993887, 7457.601081 samples/s
[Epoch 12 Batch 400] Training: accuracy=0.993940, 5548.167941 samples/s
[Epoch 12 Batch 500] Training: accuracy=0.993513, 5805.506111 samples/s
[Epoch 12] Training: accuracy=0.993133
[Epoch 12] Validation: accuracy=0.974300
[Epoch 13 Batch 100] Training: accuracy=0.995347, 6616.351963 samples/s
[Epoch 13 Batch 200] Training: accuracy=0.995423, 5502.314110 samples/s
[Epoch 13 Batch 300] Training: accuracy=0.994751, 5504.552673 samples/s
[Epoch 13 Batch 400] Training: accuracy=0.994065

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [12]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-11-15-14-37-03-872
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-11-15-14-37-03-872


--------------------------------------------------------------!

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [19]:
from IPython.display import HTML
HTML(open("input.html").read())

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [20]:
response = predictor.predict(data)
print int(response)

3


# Challenge
- Use your convolutoinal network from previous lab and port them into Amazon SageMaker

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)